In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from sklearn.preprocessing import LabelEncoder
import gym
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import csr_matrix


# Data preprocessing
# Load the Yoochoose dataset and preprocess it to create session-based sequences of interactions
data = pd.read_csv('yoochoose_dataset/filtered_clicks.dat',
                   names=['session_id', 'timestamp', 'item_id', 'category'],
                   dtype={'session_id': 'int64', 'timestamp': 'str', 'item_id': 'int64', 'category': 'int64'},
                   parse_dates=['timestamp'])

# Create item and session maps
item_map = dict(zip(np.unique(data.item_id), range(len(np.unique(data.item_id)))))
session_map = dict(zip(np.unique(data.session_id), range(len(np.unique(data.session_id)))))

# Map item and session IDs
data['item_id'] = data['item_id'].map(item_map)
data['session_id'] = data['session_id'].map(session_map)

# Sort by session and timestamp
data = data.sort_values(['session_id', 'timestamp'])

# Create next item and session columns
data['next_item_id'] = data.groupby('session_id')['item_id'].shift(-1)
data['next_session_id'] = data.groupby('session_id')['session_id'].shift(-1)
data = data.dropna()

# Convert data to numpy arrays
session_ids = data['session_id'].values.astype('int32')
item_ids = data['item_id'].values.astype('int32')
next_item_ids = data['next_item_id'].values.astype('int32')
next_session_ids = data['next_session_id'].values.astype('int32')
timestamps = data['timestamp'].values

# Create a directed graph
graph = nx.DiGraph()

# Add nodes to the graph
graph.add_nodes_from(item_map.values())

# Add edges to the graph
for session_id, items in tqdm(data.groupby('session_id')['item_id']):
    items = items.values.tolist()
    for i in range(len(items)-1):
        src, dst = items[i], items[i+1]
        graph.add_edge(src, dst)
        
        
# Create dense feature matrix
num_items = len(item_map)
features = np.eye(num_items, dtype='float32')[item_ids]

# Create adjacency matrix
adj_matrix = nx.adjacency_matrix(graph, nodelist=range(num_items))
adj_matrix = csr_matrix(adj_matrix)
adj_indices = np.array(adj_matrix.nonzero()).T
# Get adjacency matrix in a flat list
adj_list = [adj_matrix[i, j] for i, j in adj_indices]
# Convert adjacency list to a sparse tensor
adj_values = tf.constant(adj_list, dtype=tf.float32)
num_nodes = adj_matrix.shape[0]

100%|██████████| 1756/1756 [00:00<00:00, 93727.46it/s]


In [9]:
class GNNModel(tf.keras.Model):
    def __init__(self, num_items, num_features, hidden_units):
        super(GNNModel, self).__init__()
        self.num_items = num_items
        self.num_features = num_features
        self.hidden_units = hidden_units

        self.input_layer = Input(shape=(1,), dtype=tf.int64)
        self.embedding_layer = layers.Embedding(input_dim=self.num_items, output_dim=self.num_features)(self.input_layer)
        self.embedding_layer = layers.Reshape(target_shape=(self.num_features,))(self.embedding_layer)

        self.inputs_layer = Input(shape=(None, self.num_features))
        self.adj_dense_layer = layers.Dense(units=self.num_items, activation='sigmoid')(self.embedding_layer)
        self.adjacency_layer = tf.expand_dims(self.adj_dense_layer, axis=0)
        self.features_layer = self.inputs_layer[:, :, 1:]
        self.features_reshape_layer = layers.Reshape(target_shape=(tf.shape(self.inputs_layer)[1], self.num_features-1))(self.features_layer)
        self.concat_layer = layers.Concatenate(axis=-1)([self.embedding_layer, self.features_reshape_layer])
        self.gcn_layer = layers.Dense(units=self.hidden_units, activation='relu')(self.concat_layer)
        self.flatten_layer = layers.Flatten()(self.gcn_layer)
        self.outputs_layer = layers.Dense(units=self.num_items, activation='softmax')(self.flatten_layer)

        self.model = Model(inputs=[self.input_layer, self.inputs_layer], outputs=self.outputs_layer)
    
    def compile_model(self, learning_rate=0.001):
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



def contrastive_loss(y_true, y_pred):
    positive_indices = tf.where(y_true > 0)
    negative_indices = tf.where(y_true == 0)
    positive_scores = tf.gather_nd(y_pred, positive_indices)
    negative_scores = tf.gather_nd(y_pred, negative_indices)
    positive_scores = tf.expand_dims(positive_scores, axis=1)
    loss = tf.reduce_mean(tf.maximum(0.0, 1.0 - positive_scores + negative_scores))
    return loss


In [18]:
# Define hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001
embedding_dim = 32
num_features = features.shape[1]

# Split data into training and validation sets
train_indices, val_indices = train_test_split(np.arange(len(session_ids)), test_size=0.2, random_state=42)

# Define model
model = GNNModel(num_items, num_features, embedding_dim)
model.compile_model(learning_rate)

# Train model
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    np.random.shuffle(train_indices)
    pbar = tqdm(range(0, len(train_indices), batch_size))
    for i in pbar:
        batch_indices = train_indices[i:i+batch_size]
        batch_session_ids = session_ids[batch_indices]
        batch_item_ids = item_ids[batch_indices]
        batch_features = features[batch_item_ids]
        batch_next_item_ids = next_item_ids[batch_indices]
        batch_labels = np.zeros((len(batch_indices), num_items))
        for j, next_item_id in enumerate(batch_next_item_ids):
            if next_item_id is not None:
                batch_labels[j, next_item_id] = 1
        batch_labels = np.argmax(batch_labels, axis=1)

        # Create sparse tensor from adjacency matrix
        batch_adj_values = adj_values[batch_item_ids]
        batch_adj_indices = adj_indices[batch_item_ids]
        batch_adj_indices[:, 0] -= i
        batch_adj_indices[:, 1] -= i
        batch_adj_indices += np.array([np.zeros_like(batch_adj_indices[:, 0]), batch_indices]).T.reshape(-1, 2) * num_nodes
        batch_adj_indices = np.transpose(batch_adj_indices)
        batch_adj_shape = (batch_size, num_nodes, num_nodes)
        batch_adj_tensor = tf.sparse.SparseTensor(batch_adj_indices, batch_adj_values, batch_adj_shape)

        # Train on batch
        loss, accuracy = model.model.train_on_batch([batch_item_ids, batch_features, batch_adj_tensor], batch_labels)
        pbar.set_description(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

    # Evaluate model on validation set
    val_session_ids = session_ids[val_indices]
    val_item_ids = item_ids[val_indices]
    val_features = features[val_item_ids]
    val_next_item_ids = next_item_ids[val_indices]
    val_labels = np.zeros((len(val_indices), num_items))
    for j, next_item_id in enumerate(val_next_item_ids):
        if next_item_id is not None:
            val_labels[j, next_item_id] = 1
    val_labels = np.argmax(val_labels, axis=1)

    # Create sparse tensor from adjacency matrix
    val_adj_values = adj_values[val_item_ids]
    val_adj_indices = adj_indices[val_item_ids]
    val_adj_indices[:, 1] += len(session_ids)
    val_adj_indices = np.transpose(val_adj_indices)
    val_adj_shape = (len(val_indices), num_nodes, num_nodes)
    val_adj_tensor = tf.sparse.SparseTensor(val_adj_indices, val_adj_values, val_adj_shape)

    val_loss, val_accuracy = model.model.evaluate([val_item_ids, val_features, val_adj_tensor], val_labels, verbose=0)
    print(f'Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}')

print('Training complete!')


ValueError: as_list() is not defined on an unknown TensorShape.